In [1]:
import math
import time
from dataclasses import dataclass
from typing import List, Tuple

import vrplib
from ortools.constraint_solver import pywrapcp, routing_enums_pb2


@dataclass
class VRPInstance:
    name: str
    dimension: int          # number of nodes (incl. depot)
    capacity: int           # vehicle capacity
    num_vehicles: int
    coords: List[Tuple[float, float]]
    demands: List[int]
    depot_index: int        # 0-based
    distance_matrix: List[List[int]]


def load_instance_with_vrplib(path: str) -> VRPInstance:
    """
    Load a VRPLIB CVRP instance (e.g. X-n101-k25) using vrplib,
    including the distance matrix.
    """
    inst = vrplib.read_instance(path)   # parses VRPLIB and can compute distances :contentReference[oaicite:2]{index=2}

    name = inst["name"]
    coords = inst["node_coord"]         # shape (n, 2), depot at index 0
    demands = inst["demand"]           # length n, depot demand should be 0
    depot = int(inst["depot"][0])      # already 0-based by vrplib :contentReference[oaicite:3]{index=3}
    capacity = int(inst["capacity"])
    num_vehicles = int(inst.get("vehicles", 25))  # X-n101-k25: 25 vehicles

    # Use vrplib's computed edge weights if available; otherwise, compute them.
    if "edge_weight" in inst:
        ew = inst["edge_weight"]
        # Or-Tools expects integer costs; we round to nearest int.
        distance_matrix = [[int(round(float(ew[i, j]))) for j in range(ew.shape[1])]
                           for i in range(ew.shape[0])]
    else:
        # Fallback: compute Euclidean distances and round to nearest int
        coords_list = coords.tolist()
        n = len(coords_list)
        distance_matrix = [[0] * n for _ in range(n)]
        for i in range(n):
            x_i, y_i = coords_list[i]
            for j in range(i + 1, n):
                x_j, y_j = coords_list[j]
                d = math.hypot(x_i - x_j, y_i - y_j)
                v = int(d + 0.5)
                distance_matrix[i][j] = v
                distance_matrix[j][i] = v

    coords_list = [tuple(map(float, c)) for c in coords.tolist()]
    demands_list = [int(d) for d in demands.tolist()]
    dimension = len(coords_list)

    total_demand = sum(demands_list)
    print(f"Loaded instance {name}")
    print(f"  nodes        : {dimension}")
    print(f"  vehicles     : {num_vehicles}")
    print(f"  capacity Q   : {capacity}")
    print(f"  total demand : {total_demand}")
    print(f"  max demand   : {max(demands_list)}")
    print(f"  total cap    : {num_vehicles * capacity}")
    if max(demands_list) > capacity:
        raise ValueError("Infeasible: some customer demand exceeds vehicle capacity.")
    if total_demand > num_vehicles * capacity:
        print("Warning: total demand > total vehicle capacity → infeasible model.")

    return VRPInstance(
        name=name,
        dimension=dimension,
        capacity=capacity,
        num_vehicles=num_vehicles,
        coords=coords_list,
        demands=demands_list,
        depot_index=depot,
        distance_matrix=distance_matrix,
    )



# ---------- Distance matrix ----------

def build_distance_matrix(coords: List[Tuple[float, float]]) -> List[List[int]]:
    """
    Build integer distance matrix using TSPLIB-style rounding of Euclidean distances.

    Even though the X instances say `EDGE_WEIGHT_TYPE: EUC_2D`, the literature
    typically assumes rounding to the nearest integer. :contentReference[oaicite:5]{index=5}
    """
    n = len(coords)
    dist = [[0] * n for _ in range(n)]
    for i in range(n):
        x_i, y_i = coords[i]
        for j in range(i + 1, n):
            x_j, y_j = coords[j]
            d = math.hypot(x_i - x_j, y_i - y_j)
            v = int(d + 0.5)  # round to nearest integer
            dist[i][j] = v
            dist[j][i] = v
    return dist


# ---------- OR-Tools model ----------

def solve_cvrp_with_ortools(instance: VRPInstance, time_limit_seconds: int = 60):
    """
    Standard CVRP model:
      - capacity constraints via a 'Capacity' dimension
      - objective = total distance
      - metaheuristic = GUIDED_LOCAL_SEARCH
    """
    distance_matrix = build_distance_matrix(instance.coords)

    data = {
        "distance_matrix": distance_matrix,
        "demands": instance.demands,
        "vehicle_capacities": [instance.capacity] * instance.num_vehicles,
        "num_vehicles": instance.num_vehicles,
        "depot": instance.depot_index,
    }

    manager = pywrapcp.RoutingIndexManager(
        len(distance_matrix),
        data["num_vehicles"],
        data["depot"],
    )
    routing = pywrapcp.RoutingModel(manager)

    # Distance callback
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return distance_matrix[from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Demand callback
    def demand_callback(from_index):
        from_node = manager.IndexToNode(from_index)
        return data["demands"][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)

    # Capacity dimension (cumulative load)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # no slack
        data["vehicle_capacities"],
        True,  # start cumulative load at 0
        "Capacity",
    )

    # Search parameters (from OR-Tools docs) :contentReference[oaicite:6]{index=6}
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    )
    search_parameters.time_limit.FromSeconds(time_limit_seconds)
    search_parameters.log_search = True  # helpful for benchmarking

    print(
        f"\nSolving instance {instance.name} "
        f"({instance.dimension-1} customers, {instance.num_vehicles} vehicles, Q={instance.capacity})"
    )
    start = time.perf_counter()
    solution = routing.SolveWithParameters(search_parameters)
    wall_time = time.perf_counter() - start

    if solution is None:
        print("No solution found (model likely infeasible or search too constrained).")
        print(f"Wall time: {wall_time:.3f} seconds")
        return

    print_solution(instance, data, manager, routing, solution, wall_time)


def print_solution(
    instance: VRPInstance,
    data,
    manager: pywrapcp.RoutingIndexManager,
    routing: pywrapcp.RoutingModel,
    solution: pywrapcp.Assignment,
    wall_time: float,
):
    capacity_dim = routing.GetDimensionOrDie("Capacity")

    print("\n========== Solution ==========")
    print(f"Instance: {instance.name}")
    print(f"Objective (total distance): {solution.ObjectiveValue()}")
    print(f"Wall time: {wall_time:.3f} seconds\n")

    total_distance = 0
    total_load = 0

    for vehicle_id in range(data["num_vehicles"]):
        if not routing.IsVehicleUsed(solution, vehicle_id):
            continue

        index = routing.Start(vehicle_id)
        route_distance = 0

        print(f"Route for vehicle {vehicle_id}:")
        while not routing.IsEnd(index):
            node = manager.IndexToNode(index)
            load = solution.Value(capacity_dim.CumulVar(index))
            print(f"  Node {node:3d}  Load({load:3d})")
            prev_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(prev_index, index, vehicle_id)

        node = manager.IndexToNode(index)
        load = solution.Value(capacity_dim.CumulVar(index))
        print(f"  Node {node:3d}  Load({load:3d})")

        print(f"  Route distance: {route_distance}")
        print(f"  Route load    : {load}\n")

        total_distance += route_distance
        total_load += load

    print("========== Summary ==========")
    print(f"Total distance of all routes: {total_distance}")
    print(f"Total load delivered:        {total_load}")
    print("=============================\n")

In [3]:
path = "Uchoa_et_al_2014/X-n101-k25.vrp"

instance = load_instance_with_vrplib(path)
sol = solve_cvrp_with_ortools(instance, time_limit_seconds=60)

Loaded instance X-n101-k25
  nodes        : 101
  vehicles     : 25
  capacity Q   : 206
  total demand : 5147
  max demand   : 100
  total cap    : 5150

Solving instance X-n101-k25 (100 customers, 25 vehicles, Q=206)
No solution found (model likely infeasible or search too constrained).
Wall time: 60.001 seconds


In [4]:
sol